## 1. The ten best-selling video games - Project form DataCamp

<p>This is a guided project from DataCamp, and just meant as a way to briefly show my SQL skills.
In this project, I'll explore the top 400 best-selling video games created between 1977 and 2020, by comparing a dataset on game sales with critic and user reviews, to determine whether or not video games have improved as the gaming market has grown.</p>
<p>The database contains two tables; each table is limited to 400 rows for this project, but the complete dataset with over 13,000 games is on <a href="https://www.kaggle.com/holmjason2/videogamedata">Kaggle</a>. </p>
<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Gaming platform</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Game publisher</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Game developer</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Number of copies sold (millions)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Release year</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Critic score according to Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>User score according to Metacritic</td>
</tr>
</tbody>
</table>

In [1]:
%%sql
postgresql:///games

-- Find all information for the top ten best-selling games
-- Order the results from best-selling game down to tenth best-selling

SELECT *
FROM game_sales
ORDER BY games_sold
LIMIT(10);

10 rows affected.


game,platform,publisher,developer,games_sold,year
Namco Museum: 50th Anniversary for PS2,PS2,Namco,Digital Eclipse,3.98,2005
The Legend of Zelda: Oracle of Ages / Seasons for GBC,GBC,Nintendo,Capcom,3.99,2001
The Elder Scrolls V: Skyrim for PC,PC,Bethesda Softworks,Bethesda Game Studios,3.99,2011
Fallout 3 for PS3,PS3,Bethesda Softworks,Bethesda Game Studios,4.00,2008
EyeToy Play for PS2,PS2,Sony Computer Entertainment,SCEE London Studio,4.00,2003
Diablo II for PC,PC,Blizzard Entertainment,Blizzard North,4.00,2000
Just Dance for Wii,Wii,Ubisoft,Ubisoft Paris,4.00,2009
Arma III for PC,PC,Bohemia Interactive,Bohemia Interactive,4.00,2013
Disney's Aladdin for GEN,GEN,Sega,Virgin Interactive,4.00,1993
Monster Hunter: World for PS4,PS4,Capcom,Capcom,4.00,2018


## 2. Missing review scores
<p>The best-selling video games seems to be released between 1985 to 2017.
<p>Use data from the <code>reviews</code> table to gain more insight on the best years for video games. 
<p>One big shortcoming of the dataset, is that there is not any <code>reviews</code> data for some of the games on the <code>game_sales</code> table. </p>

In [3]:
%%sql 

-- Join games_sales and reviews
-- Find the number of games where both critic_score and user_score are null
SELECT COUNT(g.game)
FROM game_sales g
LEFT JOIN reviews r
ON g.game = r.game
WHERE critic_score IS NULL AND user_score IS NULL;

 * postgresql:///games
1 rows affected.


count
31


## 3. Years that video game critics loved
<p>It looks like a little less than ten percent of the games on the <code>game_sales</code> table don't have any reviews data. I'll continue exploring the data, but the missing reviews data is a good thing to keep in mind when evaluating results from queries later on. </p>
<p> First, let's look at the best year for video games - according to the critics.

In [6]:
%%sql

-- Select release year and average critic score for each year, rounded and aliased
-- Join the game_sales and reviews tables
-- Group by release year
-- Order the data from highest to lowest avg_critic_score and limit to 10 results

SELECT year, ROUND(AVG(critic_score),2) as avg_critic_score
FROM game_sales
inner join reviews
ON game_sales.game = reviews.game
GROUP BY game_sales.year
ORDER BY avg_critic_score DESC
LIMIT(10);


 * postgresql:///games
10 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
1998,9.32
2020,9.20
1993,9.10
1995,9.07
2004,9.03
1982,9.00
2002,8.99
1999,8.93


## 4. Was 1982 really that great?
<p>The range of great years according to critic reviews goes from 1982 until 2020.
<p>Some of those <code>avg_critic_score</code> values look like suspiciously round numbers for averages. The value for 1982 looks especially fishy. Maybe there weren't a lot of video games released in certain years. </p>
<p>Let's update the query and find out whether 1982 really was such a great year for video games.</p>

In [13]:
%%sql 

-- Paste your query from the previous task; update it to add a count of games released in each year called num_games
-- Update the query to only returns years that have more than four reviewed games

SELECT year, ROUND(AVG(critic_score),2) as avg_critic_score, COUNT(game_sales.game) AS num_games
FROM game_sales
inner join reviews
ON game_sales.game = reviews.game
GROUP BY game_sales.year
HAVING COUNT(game_sales.game) > 4
ORDER BY avg_critic_score DESC
LIMIT(10);

 * postgresql:///games
10 rows affected.


year,avg_critic_score,num_games
1998,9.32,10
2004,9.03,11
2002,8.99,9
1999,8.93,11
2001,8.82,13
2011,8.76,26
2016,8.67,13
2013,8.66,18
2008,8.63,20
2017,8.62,13


## 5. Years that dropped off the critics' favorites list
<p> The <code>num_games</code> column now reflects years that had quite a few well-reviewed games rather than just one or two hits. Which years dropped off the list due to having four or fewer reviewed games?
<p> There are tables with the results of the previous two queries:</p>
<h3 id="top_critic_years"><code>top_critic_years</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>
<h3 id="top_critic_years_more_than_four_games"><code>top_critic_years_more_than_four_games</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Year of video game release</td>
</tr>
<tr>
<td style="text-align:left;"><code>num_games</code></td>
<td>int</td>
<td>Count of the number of video games released in that year</td>
</tr>
<tr>
<td style="text-align:left;"><code>avg_critic_score</code></td>
<td>float</td>
<td>Average of all critic scores for games released in that year</td>
</tr>
</tbody>
</table>

In [22]:
%%sql 
-- Select the year and avg_critic_score for those years that dropped off the critics' favorites list
-- Order the results from highest to lowest avg_critic_score

SELECT year, avg_critic_score
FROM top_critic_years
EXCEPT
SELECT year, avg_critic_score
FROM top_critic_years_more_than_four_games
ORDER BY avg_critic_score DESC;

 * postgresql:///games
6 rows affected.


year,avg_critic_score
1990,9.80
1992,9.67
2020,9.20
1993,9.10
1995,9.07
1982,9.00


## 6. Years video game players loved
<p>It looks like the early 1990s might merit consideration as the golden age of video games based on <code>critic_score</code> alone.
<p>Let's look at the opinions players. Create a query very similar to the one used in Task Four, except this will look at <code>user_score</code> averages by year rather than <code>critic_score</code> averages.</p>

In [26]:
%%sql 

-- Select year, an average of user_score, and a count of games released in a given year, aliased and rounded
-- Include only years with more than four reviewed games; group data by year
-- Order data by avg_user_score, and limit to ten results

SELECT game_sales.year, COUNT(game_sales.game) AS num_games, ROUND(AVG(reviews.user_score),2) AS avg_user_score
FROM game_sales
INNER JOIN reviews
ON game_sales.game = reviews.game
GROUP BY game_sales.year
HAVING COUNT(game_sales.game) > 4
ORDER BY avg_user_score DESC
LIMIT(10);

 * postgresql:///games
10 rows affected.


year,num_games,avg_user_score
1997,8,9.50
1998,10,9.40
2010,23,9.24
2009,20,9.18
2008,20,9.03
1996,5,9.00
2005,13,8.95
2006,16,8.95
2000,8,8.80
2002,9,8.80


## 7. Years that both players and critics loved
<p> Here's a list of the top ten years according to both critic reviews and user reviews. Are there any years that showed up on both tables?


In [27]:
%%sql 

-- Select the year results that appear on both tables
-- Select the year results that appear on both tables

SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games;

 * postgresql:///games
3 rows affected.


year
1998
2008
2002


## 8. Sales in the best video game years
There are three years that both users and critics agreed were in the top ten.
While there are many other ways of measuring what the best years for video games are,  let's stick with these years for now. We know that critics and players liked these years, but how were the sales?

In [29]:
%%sql 

-- Select year and sum of games_sold, aliased as total_games_sold; order results by total_games_sold descending
-- Filter game_sales based on whether each year is in the list returned in the previous task

SELECT game_sales.year, SUM(game_sales.games_sold) AS total_games_sold
FROM game_sales
WHERE game_sales.year IN (SELECT year
FROM top_user_years_more_than_four_games
INTERSECT
SELECT year
FROM top_critic_years_more_than_four_games)
GROUP BY game_sales.year
ORDER BY total_games_sold DESC;

 * postgresql:///games
3 rows affected.


year,total_games_sold
2008,175.07
1998,101.52
2002,58.67
